In [1]:
from pathlib import Path
import numpy as np
from PIL import Image
from IPython.display import display
import cv2 as cv
import numpy as np
import json
import os
import xmltodict
from datetime import datetime
from matplotlib import pyplot as plt

In [2]:
json_path = '/home/adichoudhary/CSE523/vehicle-ReId/repo/data/dataset_1.json'

In [3]:
with open(json_path, "r") as read_file:
    data = json.load(read_file)

In [4]:
data.keys()

dict_keys(['Set01', 'Set02', 'Set03', 'Set04', 'Set05'])

In [5]:
for k in data.keys():
    print (len(data[k]))

102810
84092
96389
63827
76343


## Extract Frames

In [11]:
def get_images(orig_image, patch):
    
#     print ("Orig image - ", orig_image)
#     print ("patch - ", patch)


    ## search images 
    img = cv.imread(orig_image,0)
#     img2 = img.copy()
    template = cv.imread(patch,0)
    w, h = template.shape[::-1]

    # method = 'cv.TM_CCOEFF_NORMED'
#     display(img)
#     display(template)
    # Apply template Matching
    res = cv.matchTemplate(img,template,cv.TM_CCOEFF_NORMED)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
#     print (max_val)
    if max_val < 0.95:
        print(max_val)
        print(orig_image)
        print(patch)
        print ()
        
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    w_2 = int(w/2)
    h_2 = int(h/2)
#     print (top_left)
#     print(bottom_right)
    
    # all surrounding
    bdry_top_left_all = (max(0,max_loc[0]-w_2), max(0,max_loc[1]-h_2))
    # without decreasing height
    bdry_top_left = (max(0,max_loc[0]-w_2), max_loc[1])
    
    #Original image size - (1920, 1080)
    bdry_bottom_right = (min(bottom_right[0]+w_2,1920), 
                         min(bottom_right[1]+h_2,1080))
#     print ("Boundary ")
#     print (bdry_top_left_all)
#     print (bdry_bottom_right)
    
    # Now cropping from the above results
    im = Image.open(orig_image)
    im_crop_center = im.crop((top_left[0], top_left[1], bottom_right[0], bottom_right[1]))

    # To delete the between patch
    rect_size = (w, h)
    rect_pos = top_left
    im2 = im#.copy()
    
    rect = Image.new("RGBA", rect_size, (255, 255, 255, 0))
    im2.paste(rect, rect_pos)

    im_crop_all = im2.crop((bdry_top_left_all[0], bdry_top_left_all[1],
                       bdry_bottom_right[0], bdry_bottom_right[1]))
    
    im_crop = im2.crop((bdry_top_left[0], bdry_top_left[1],
                       bdry_bottom_right[0], bdry_bottom_right[1]))
    
    return [im_crop_center, im_crop, im_crop_all]
    
    

In [8]:
# car_1_img.split('.')[0]

In [6]:
len(data['Set02'])

84092

In [13]:
for num in range(2,3):
    
    parent_dir = "/home/adichoudhary/CSE523/vehicle-ReId/repo/data_new/dataset/"
    
    set_num = "Set0"+str(num)
    i=0
    car_str = 'classes_carros'
    sur_str = 'classes_surr'
    surall_str = 'classes_surrall'
    class_li = [car_str, sur_str, surall_str]
    start=datetime.now()
    for row in data[set_num]:
    
        label = row[4]
        #Camera 1
        car_1 = row[1][0]
        #Camera 2
        car_2 = row[3][0]

        car_1_name = car_1.split('/')[-2]
        car_1_img = car_1.split('/')[-1]
        car_1_img = int(car_1_img.split('.')[0])+1
        
        car_2_name = car_2.split('/')[-2]
        car_2_img = car_2.split('/')[-1]
        car_2_img = int(car_2_img.split('.')[0])+1
        
#         print(car_1,car_1_img)
#         print(car_2,car_2_img)
#         print(label)
        
        car_1_orig_img = '/data/adichoudhary/frames/Camera1/Set0'+str(num)+'/'+str(car_1_img)+'.jpg'

        patch_1 = '/home/adichoudhary/CSE523/vehicle-ReId/repo/'+car_1
        
        
        car_2_orig_img = '/data/adichoudhary/frames/Camera2/Set0'+str(num)+'/'+str(car_2_img)+'.jpg'

        patch_2 = '/home/adichoudhary/CSE523/vehicle-ReId/repo/'+car_2
        
        res_1 = get_images(car_1_orig_img, patch_1)
        
#         for r in res_1:
#             display(r)
        
        res_2 = get_images(car_2_orig_img, patch_2)
        
#         for r in res_2:
#             display(r)
        # check if all 3 images already present - Continue
        
        
        #Save these 3 images per Car to respective directories

        for cl in range(len(class_li)):
            path_1 = os.path.join(parent_dir, "Camera1", set_num, class_li[cl], car_1_name ) 
            path_2 = os.path.join(parent_dir, "Camera2", set_num, class_li[cl], car_2_name )

    #         os.mkdir(path_1)
            Path(path_1).mkdir(parents=True, exist_ok=True)
            Path(path_2).mkdir(parents=True, exist_ok=True)
            
            img_1_path = path_1+"/"+str(car_1_img-1)+".png"
            res_1[cl].save(img_1_path)
            
            img_2_path = path_2+"/"+str(car_2_img-1)+".png"
            res_2[cl].save(img_2_path)
            
#             print (img_1_path)
#             print (img_2_path)
#             print ()
            
        if i%1000 == 0:
            print ('Time at line ',i,' - ',datetime.now()-start)
            start=datetime.now()
        i+=1
    
    


Time at line  0  -  0:00:01.074811
Time at line  1000  -  0:22:20.718110
Time at line  2000  -  0:20:25.528605
Time at line  3000  -  0:23:47.563153
Time at line  4000  -  0:21:41.580538
Time at line  5000  -  0:21:02.395130
Time at line  6000  -  0:20:45.429804
Time at line  7000  -  0:20:06.041667
Time at line  8000  -  0:23:24.676408
Time at line  9000  -  0:22:13.887361
Time at line  10000  -  0:22:11.309904
Time at line  11000  -  0:18:53.371878
Time at line  12000  -  0:22:09.401393
Time at line  13000  -  0:26:49.891451
Time at line  14000  -  0:34:06.962851
Time at line  15000  -  0:25:17.009701
Time at line  16000  -  0:19:33.954979
Time at line  17000  -  0:19:25.740454
Time at line  18000  -  0:17:42.185300
Time at line  19000  -  0:17:47.829743
Time at line  20000  -  0:17:42.897754
Time at line  21000  -  0:19:24.628879
Time at line  22000  -  0:20:04.416756
Time at line  23000  -  0:18:30.239948
Time at line  24000  -  0:19:28.085586
Time at line  25000  -  0:20:06.057416